<a href="https://colab.research.google.com/github/caiodavic/Processamento-de-Linguagem-Natural-2022.1/blob/main/Agrupamento_com_k_means_PLN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer, SnowballStemmer, WordNetLemmatizer
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# Leitura de Dados
- Vamos utilizar sátiras políticas do sensacionalista.

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/caiolibanio/atividade_NLP/master/csv_satiras_politicas.csv")
df.head()

In [ ]:
tokenizer = nltk.tokenize.RegexpTokenizer('\w+')
stop_words = stopwords.words("portuguese")
stop_words.extend(['nao','diz'])
def pre_process(sentence):
  
  tokens = tokenizer.tokenize(sentence)
  filtered = ' '.join([i for i in tokens if i not in stop_words])
  return filtered

In [ ]:
titles = df['title'].apply(pre_process)
texts = df['text'].apply(pre_process)
small_df = pd.DataFrame({'title':titles,'texts':texts})

# Extração de Features - TF-IDF
- Função para gerar as matrizes TF-IDF a partir da coleção de documentos textuais passados como entrada

In [ ]:
def buildTFIDFVectorizer(docs):
  vect = TfidfVectorizer()
  matrix = vect.fit_transform(docs)
  return matrix, vect

In [ ]:
m, v = buildTFIDFVectorizer(texts)
tf_idf = pd.DataFrame(data = m.toarray(), columns = v.get_feature_names_out())

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
fig = plt.figure(figsize= (30,10))
ax = fig.add_subplot(1, 2, 1)
pca = PCA(n_components=2)
scatter_plot_points = pca.fit_transform(m.toarray())

ax.scatter(scatter_plot_points[:, 0], scatter_plot_points[:,1])

# Executando o k-means
- Vamos executar o k-means com o número k já definido, k=3.

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(m)
groups = kmeans.predict(m)
groups

In [ ]:
small_df['group'] = groups

In [ ]:
def plot(m):
  fig = plt.figure(figsize= (30,10))
  ax = fig.add_subplot(1, 2, 1)
  pca = PCA(n_components=3, random_state= 0)
  reduced_features = pca.fit_transform(m.toarray())
  reduced_cluster_centers = pca.transform(kmeans.cluster_centers_)
  ax.scatter(reduced_features[:,0], reduced_features[:,1], c=kmeans.predict(m))
  ax.scatter(reduced_cluster_centers[:, 0], reduced_cluster_centers[:,1])
  plt.show()

In [ ]:
plot(m)

# Analisando as palavras mais frequentes de cada grupo
- A partir de agora vamos analisar com um histograma e um wordCloud as palavras mais frequentes de cada cluster

In [ ]:
def counting_words(df,predict):
  labels = np.unique(predict)
  couting = []
  for i in labels:
    mask = df['group'] == i
    next_to_cluster = df[mask]
    a = next_to_cluster['title'].str.split()
    freq_word = []
    for i in a:
      freq_word = freq_word + i    
    couting.append(nltk.FreqDist(freq_word))

  return couting  

In [ ]:
import matplotlib.pyplot as plt 
def plot_horizontal(counting):
  
  for i in range(len(counting)):
    y =[j[0] for j in counting[i].most_common(5)] 
    x=[j[1] for j in counting[i].most_common(5)] 

    
    plt.barh(y, x) 
    plt.ylabel("words") 
    plt.xlabel("frequency")  
    plt.title(f'5 most frequent words from cluster {i}') 
    plt.show() 

In [ ]:
couting_aux = counting_words(small_df,groups)

In [ ]:
plot_horizontal(couting_aux)

In [ ]:
from wordcloud import WordCloud

In [ ]:
def generate_word_clouds(centroids):
    wordcloud = WordCloud(max_font_size=100, background_color = 'white')
    for i in range(0, len(centroids)):
        centroid_dict = centroids[i]        
        wordcloud.generate_from_frequencies(centroid_dict)

        plt.figure()
        plt.title(f'Cluster {i}')
        plt.imshow(wordcloud)
        plt.axis("off")
        plt.show()

In [ ]:
generate_word_clouds(counting_words(small_df,groups))

# Escolhendo o K no K-Means
- Precisamos de alguma heurística que nos ajude a escolher o K com um pouco mais de critério.
- Vamos utilizar o método do Cotovelo.

## O método do cotovelo

O método mais comum é usar uma métrica de heterogeneidade dentro dos clusters. Com uma métrica desse tipo, uma solução com k maior sempre será mais homogênea. O método então consiste de examinar vários valores de $k$ e *escolher um valor a partir do qual* **não há ganho substancial em aumentar k**.

Uma medida de heterogeneidade dos grupos é o SSD: 

SSD = $\sum{(x_i - c_i)^2}$

Professor Nazareno Andrade

In [ ]:
from tqdm import tqdm

In [ ]:
quality = pd.DataFrame(columns = ['k', 'ssd'])
for k in tqdm(range(1,5)):
  kmeans = KMeans(n_clusters=k, n_init=20)
  clsuter_labels = kmeans.fit_predict(m)
  quality = quality.append({'k': k, 'ssd' : kmeans.inertia_}, ignore_index=True)

quality.head(10)